In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
sns.set()

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
#import datetime as dt

import time
from datetime import datetime
from datetime import date
from datetime import timedelta
import scipy
import scipy.stats
import pylab
import statsmodels.api as sm
from itertools import cycle

from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score 
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler

import statsmodels.tsa.stattools as sts
import statsmodels.graphics.tsaplots as sgt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima.arima import auto_arima
from statsmodels.tsa.stattools import adfuller
from pmdarima.arima import OCSBTest
from arch import arch_model
from scipy.stats.distributions import chi2 

import yfinance
from yahoo_fin.stock_info import get_data
from Time_Series_Functions import get_close_column, get_volume_column, create_ticker_table
import warnings
warnings.filterwarnings('ignore')

C:\Users\Eric\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
C:\Users\Eric\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


In [2]:
bitcoin_df = create_ticker_table()

Enter a ticker from Yahoo! Finance. Enter "No" to continue: BTC-USD
Enter a ticker from Yahoo! Finance. Enter "No" to continue: no
Do you want daily or business day data? Enter 'd' for daily and 'b' for business: b

********** Table Head **********


,BTC-USD_Close,BTC-USD_Volume,BTC-USD_Return
2017-01-02,1021.750000,222184992,0.000000
2017-01-03,1043.839966,185168000,2.161974
2017-01-04,1154.729980,344945984,10.623277
2017-01-05,1013.380005,510199008,-12.240955
2017-01-06,902.200989,351876000,-10.971108
2017-01-09,902.828003,141876992,0.069498
2017-01-10,907.679016,115808000,0.537313
2017-01-11,777.757019,310928992,-14.313650
2017-01-12,804.833984,222326000,3.481417
2017-01-13,823.984009,168968000,2.379376


********** Table Tail **********


,BTC-USD_Close,BTC-USD_Volume,BTC-USD_Return
2022-04-20,41374.378906,27819532341,-0.309307
2022-04-21,40527.363281,35372786395,-2.047198
2022-04-22,39740.320312,28011716745,-1.942004
2022-04-25,40458.308594,35445730570,1.806700
2022-04-26,38117.460938,34569088416,-5.785827
2022-04-27,39241.121094,30981015184,2.947888
2022-04-28,39773.828125,33903704907,1.357522
2022-04-29,38609.824219,30882994649,-2.926557
2022-05-02,38529.328125,32922642426,-0.208486
2022-05-03,37750.453125,27326943244,-2.021512


In [11]:

date_series = pd.Series(bitcoin_df.index.tolist())
bitcoin_df['Date'] = date_series.dt.to_pydatetime()

In [12]:
bitcoin_df

,BTC-USD_Close,BTC-USD_Volume,BTC-USD_Return,Date
2017-01-02,1021.750000,222184992,0.000000,2017-01-02
2017-01-03,1043.839966,185168000,2.161974,2017-01-03
2017-01-04,1154.729980,344945984,10.623277,2017-01-04
2017-01-05,1013.380005,510199008,-12.240955,2017-01-05
2017-01-06,902.200989,351876000,-10.971108,2017-01-06
...,...,...,...,...
2022-05-04,39698.371094,36754404490,5.159986,2022-05-04
2022-05-05,36575.140625,43106256317,-7.867402,2022-05-05
2022-05-06,36040.921875,37795577489,-1.460606,2022-05-06
2022-05-09,30296.953125,63355494961,-15.937352,2022-05-09
